# Préparation des données

## Les  imports

In [20]:
import zipfile
import xml.sax
import csv
import glob
import logging
import shutil
import os

In [21]:
logging.basicConfig(level='INFO')

In [22]:
## Modifier le WorkDirectory
wdir = '/tmp/gas'
os.chdir(wdir)
print('current work dir is:', os.getcwd())

current work dir is: /tmp/gas


## Copie des XML dans le répertoire **'workDirectory'**

In [4]:
files = glob.glob('/home/gisEns/jcartign/datasets/PrixC*.zip')
for f in files:
    f_name = f[-30:] 
    if not f_name in os.listdir('.'):
        shutil.copy(f, '.')

## Création des CSV

In [23]:
# Création du CSV qui contient les données des points de ventes
pdvcsvfile = open('pdv.csv', 'w')
pdvnames   = ["annee",
              "id",
              "latitude",
              "longitude",
              "cp",
              "pop",
              "ouverture_debut",
              "ouverture_fin",
              "ouverture_saufjour",
              "adresse",
              "ville"]
pdv_writer = csv.DictWriter(pdvcsvfile, fieldnames=pdvnames)
pdv_writer.writeheader()

# Création du CSV qui contient les données des ruptures
rupturescsvfile = open('ruptures.csv', 'w')
rupturesnames = ["rupture_id",
                 "rupture_nom",
                 "rupture_debut",
                 "rupture_fin"]
ruptures_writer = csv.DictWriter(rupturescsvfile, fieldnames=rupturesnames)
ruptures_writer.writeheader()

# Création du CSV qui contient les données des fermetures
fermeturescsvfile = open('fermetures.csv', 'w')
fermeturesnames = ["fermeture_type",
                   "fermeture_debut",
                   "fermeture_fin"]
fermetures_writer = csv.DictWriter(fermeturescsvfile, fieldnames=fermeturesnames)
fermetures_writer.writeheader()


# Création du CSV qui contient les données des prix
prixcsvfile = open('prix.csv', 'w')
prixnames   = ["annee",
              "id_pdv",
              "nom",
              "id",
              "maj",
              "valeur"]
prix_writer = csv.DictWriter(prixcsvfile, fieldnames=prixnames)
prix_writer.writeheader()

# Création du CSV qui contient les données des services
servicescsvfile = open('services.csv', 'w')
servicesnames   = ["annee",
                   "id_pdv",
                   "service"]
services_writer = csv.DictWriter(servicescsvfile, fieldnames=servicesnames)
services_writer.writeheader()


# Initialisation des dictionnaire des lignes temporiaires à inserer dans les csv
pdv_row      = dict.fromkeys(pdvnames)
prix_row     = dict.fromkeys(prixnames)
services_row = dict.fromkeys(servicesnames)

## le SaxParserHandler

In [24]:
class PdvHandler(xml.sax.ContentHandler):
    def __init__(self, annee): # on passe l'année en parametre au constructeur
        self.CurrentData = ""
        self.annee       = annee
        self.id_pdv      = ""
        self.adresse     = ""
        self.ville       = ""
        self.service     = ""
        self.isVille     = False
      
   # Call when an element starts
    def startElement(self, tag, attributes):
        self.CurrentData = tag
        # quand on trouve la balise pdv
        if tag == "pdv":
            pdv_row['annee']= self.annee       # on commence l'implementation de la ligne à inserer
                                            # dans le pdvCsv
            self.id_pdv = attributes['id']     # on garde le id du pdv pour le rajouter dans les autres 
                                            # CSV comme clé primaire
            if attributes.getNames() != []:
                #id, latitude, longitude, cp, pop
                for name in attributes.getNames():
                    pdv_row[name] = attributes.getValue(name)

        # quand on trouve la balise ouverture
        elif tag == "ville":
            self.isVille = True
        elif tag == "ouverture":
            if attributes.getNames() != []:
                #debut, fin, saufjour
                for name in attributes.getNames():
                    pdv_row["ouverture_"+name] = attributes.getValue(name)
            
        # quand on trouve la balise fermeture
        elif tag == "fermeture":
            if attributes.getNames() != []:
                #type, debut, fin
                for name in attributes.getNames():
                    pdv_row["fermeture_"+name] = attributes.getValue(name)
                
        # quand on trouve la balise rupture
        elif tag == "rupture":
            if attributes.getNames() != []:
                #id, nom, debut, fin
                for name in attributes.getNames():
                    pdv_row["rupture_"+name] = attributes.getValue(name)
                    
        # quand on trouve la balise prix
        elif tag == "prix":
            if attributes.getNames() != []:
                prix_row['annee']  = self.annee
                prix_row['id_pdv'] = self.id_pdv
                #nom, id, maj, valeur
                for name in attributes.getNames():
                    prix_row[name] = attributes.getValue(name)
        elif self.isVille:
            self.ville += tag
    # Call when an elements ends
    def endElement(self, tag):
        if self.CurrentData == "adresse":
            pdv_row["adresse"] = self.adresse
        elif self.CurrentData == "ville":
            self.isVille = False
            pdv_row["ville"] = self.ville
            self.ville = ""
        elif self.CurrentData == "service":
            services_row['annee']   = self.annee
            services_row['id_pdv']  = self.id_pdv
            services_row["service"] = self.service
            services_writer.writerow(services_row) # insertion de la ligne dans le csv
        elif self.CurrentData == "prix":
            prix_writer.writerow(prix_row)         # insertion de la ligne dans le csv
        elif tag == "pdv":
            pdv_writer.writerow(pdv_row)           # insertion de la ligne dans le csv
            self.CurrentData = ""

    # Call when a character is read
    def characters(self, content):
        if self.CurrentData == "adresse":
            self.adresse = content
        elif self.CurrentData == "ville" or self.isVille:
            self.ville += content.strip()
        elif self.CurrentData == "service":
            self.service = content
  

## Traitement des Zip et Parsing des XML dans les CSV

In [25]:
def execute():
    # Pour chaque fichier zip dans '/tmp/' et qui commence par 'PrixC' et se termine par '.zip'
    for pathZip in glob.glob('PrixC*.zip'):
        annee   = pathZip[-8:-4]
        nameXml = pathZip[-30:-3]+"xml"

        # lire le zip
        logging.debug("Opening ZIP file " + pathZip)    
        zf = zipfile.ZipFile(pathZip, 'r')
        # lire le XML
        f  = zf.open(nameXml)
        # créer le XMLReader
        parser = xml.sax.make_parser()
        # turn off namepsaces
        parser.setFeature(xml.sax.handler.feature_namespaces, 0)
        # Construire le Handler
        Handler = PdvHandler(annee)
        # Override le superHandler
        parser.setContentHandler(Handler)
        # Parser
        logging.debug("Parsing XML file " + nameXml)
        parser.parse(f)

        # fermer le zip
        zf.close()

In [26]:
execute()

## Fermeture des CSV ouverts

In [27]:
pdvcsvfile.close()
servicescsvfile.close()
prixcsvfile.close()

## Verification du nombre de ligne insérées

In [28]:
with open('pdv.csv') as pdv:
    print("# lignes pdv:", len(pdv.readlines()))
    
with open('prix.csv') as prix:
    print("# lignes prix:", len(prix.readlines()))
    
with open('services.csv') as services:
    print("# lignes services:", len(services.readlines()))

# lignes pdv: 136716
# lignes prix: 28556968
# lignes services: 962905


In [11]:
!ls

pdv.csv				PrixCarburants_annuel_2014.zip
PrixCarburants_annuel_2007.zip	PrixCarburants_annuel_2015.zip
PrixCarburants_annuel_2008.zip	PrixCarburants_annuel_2016.zip
PrixCarburants_annuel_2009.zip	PrixCarburants_annuel_2017.zip
PrixCarburants_annuel_2010.zip	PrixCarburants_annuel_2018.zip
PrixCarburants_annuel_2011.zip	PrixCarburants_annuel_2019.zip
PrixCarburants_annuel_2012.zip	prix.csv
PrixCarburants_annuel_2013.zip	services.csv
